# select conda envirnment coz of FAISS library

In [5]:
import pandas as pd

In [6]:
df=pd.read_csv('netflix_titles.csv')

In [7]:
df

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...
...,...,...,...,...,...,...,...,...,...,...,...,...
8802,s8803,Movie,Zodiac,David Fincher,"Mark Ruffalo, Jake Gyllenhaal, Robert Downey J...",United States,"November 20, 2019",2007,R,158 min,"Cult Movies, Dramas, Thrillers","A political cartoonist, a crime reporter and a..."
8803,s8804,TV Show,Zombie Dumb,NaN,NaN,NaN,"July 1, 2019",2018,TV-Y7,2 Seasons,"Kids' TV, Korean TV Shows, TV Comedies","While living alone in a spooky town, a young g..."
8804,s8805,Movie,Zombieland,Ruben Fleischer,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",United States,"November 1, 2019",2009,R,88 min,"Comedies, Horror Movies",Looking to survive in a world taken over by zo...
8805,s8806,Movie,Zoom,Peter Hewitt,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...",United States,"January 11, 2020",2006,PG,88 min,"Children & Family Movies, Comedies","Dragged from civilian life, a former superhero..."


In [8]:
def create_textual_representation(row):
    textual_representation=f"""
Title: {row['title']},
Year: {row['release_year']},
Genre: {row['listed_in']},

Description: {row['description']},"""
    return textual_representation

In [9]:
df_copy = df.copy()
df_copy['textual_representation'] = df_copy.apply(create_textual_representation, axis=1)

In [10]:
print(df_copy['textual_representation'].values[0])


Title: Dick Johnson Is Dead,
Year: 2020,
Genre: Documentaries,

Description: As her father nears the end of his life, filmmaker Kirsten Johnson stages his death in inventive and comical ways to help them both face the inevitable.,


In [11]:
import faiss
print(faiss.__version__)
import requests
import numpy as np

1.7.4


In [12]:
dim = 4096

index = faiss.IndexFlatL2(dim)

x = np.zeros((len(df_copy['textual_representation']), dim), dtype = 'float32')

In [13]:
# x

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [14]:
for i, representation in enumerate(df_copy['textual_representation']):
    if i % 200 == 0:
        print("Processing textual representation", i)
        
        res = requests.post('http://localhost:11434/api/embeddings', 
                            json={'model': 'llama2',
                            'prompt': representation,
                                }
                            )
        embedding = res.json()['embedding']
        
        x[i] = np.array(embedding)
        
index.add(x)

Processing textual representation 0
Processing textual representation 200
Processing textual representation 400
Processing textual representation 600
Processing textual representation 800
Processing textual representation 1000
Processing textual representation 1200
Processing textual representation 1400
Processing textual representation 1600
Processing textual representation 1800
Processing textual representation 2000
Processing textual representation 2200
Processing textual representation 2400
Processing textual representation 2600
Processing textual representation 2800
Processing textual representation 3000
Processing textual representation 3200
Processing textual representation 3400
Processing textual representation 3600
Processing textual representation 3800
Processing textual representation 4000
Processing textual representation 4200
Processing textual representation 4400
Processing textual representation 4600
Processing textual representation 4800
Processing textual representatio

In [15]:
faiss.write_index(index, 'index')

In [16]:
index = faiss.read_index('index')

In [24]:
df_copy[df_copy.title.str.contains('Train to Busan')]

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,textual_representation
8613,s8614,Movie,Train to Busan,Sang-ho Yeon,"Gong Yoo, Yu-mi Jung, Dong-seok Ma, Soo-an Kim, Woo Shik Choi, Sohee, Eui-sung Kim",South Korea,"March 18, 2017",2016,TV-MA,118 min,"Action & Adventure, Horror Movies, International Movies","As a zombie outbreak sweeps the country, a dad and his daughter take a harrowing train journey in an attempt to reach the only city that's still safe.","\nTitle: Train to Busan,\nYear: 2016,\nGenre: Action & Adventure, Horror Movies, International Movies,\n\nDescription: As a zombie outbreak sweeps the country, a dad and his daughter take a harrowing train journey in an attempt to reach the only city that's still safe.,"


In [25]:
favorite_movie = df_copy.iloc[8613]

In [26]:
favorite_movie

show_id                                                                                                                                                                                                                                                                                            s8614
type                                                                                                                                                                                                                                                                                               Movie
title                                                                                                                                                                                                                                                                                     Train to Busan
director                                                                                                     

In [55]:
recommend_movie = """
Title: devils,
Year: 2024,
Genre: 

Description: The Plot is a  South Korean crime thriller film directed by Lee Yo-sup, starring Gang Dong-won. An adaptation of the Hong Kong film, Accident, it tells the story of Yeong-il, a designer who orchestrates a commissioned murder as a perfect accidental death, and gets caught up in an unexpected incident
"""

In [56]:
res = requests.post('http://localhost:11434/api/embeddings', 
                            json={'model': 'llama2',
                            'prompt': recommend_movie
                                })

In [57]:
embedding = np.array([res.json()['embedding']], dtype = 'float32')

D ,I = index.search(embedding, 5)

In [58]:
# I

In [59]:
best_matches = np.array(df_copy['textual_representation'])[I.flatten()]

In [60]:
for match in best_matches:
    print('Next Movie')
    print(match)
    print()

Next Movie

Title: The Long Goodbye: The Kara Tippetts Story,
Year: 2019,
Genre: Documentaries, Faith & Spirituality,

Description: Faced with terminal breast cancer, a courageous wife and mother turns her suffering into a resilient message of faith, love and compassion.,

Next Movie

Title: Devil's Gate,
Year: 2017,
Genre: Horror Movies, Sci-Fi & Fantasy, Thrillers,

Description: Seeking a missing woman in North Dakota, an FBI agent and a sheriff focus on her religious zealot husband but discover something far more sinister.,

Next Movie

Title: Tobot,
Year: 2018,
Genre: Kids' TV, Korean TV Shows,

Description: They’re robots that can transform into cars – and the Tobots are committed to fighting villains to protect the city and root out crime.,

Next Movie

Title: The Game,
Year: 1997,
Genre: Thrillers,

Description: An aloof investment banker's life spirals into peril and paranoia after his brother gives him an odd birthday gift: the chance to play a mysterious game.,

Next Movie

T